# Prerequisites
Install the `OpenPAI` sdk from `github` and specify your cluster information in `~/.openpai/clusters.yaml`. 

And for simplicity and security, we recommand user to setup necessary information in `.openpai/defaults.json` other than shown in the example notebook. (Refer to for [README](https://github.com/microsoft/pai/blob/sdk-release-v0.4.00/contrib/python-sdk/README.md) more details.)

_Please make sure you have set default values for ***cluster-alias***. This notebook will not set them explicitly for security and privacy issue_

If not, use below commands to set them
```bash
opai set cluster-alias=<your/cluster/alias>
```

In [ ]:
%load_ext autoreload
%autoreload 2

from openpaisdk.command_line import Engine
from openpaisdk.core import ClusterList, in_job_container
from uuid import uuid4 as randstr

clusters = Engine().process(['cluster', 'list'])
default_values = Engine().process(['set'])
print(default_values)

cluster_alias = default_values["cluster-alias"]
assert cluster_alias in clusters, "please specify cluster-alias and workspace"


# Submit jobs

Now we submit jobs from 
- an existing version 1 job config file
- an existing version 2 job config file
- a hello-world command line

In [ ]:
%%writefile mnist_v1.json
{
    "jobName": "keras_tensorflow_backend_mnist",
    "image": "openpai/pai.example.keras.tensorflow:stable",
    "taskRoles": [
        {
            "name": "mnist",
            "taskNumber": 1,
            "cpuNumber": 4,
            "memoryMB": 8192,
            "gpuNumber": 1,
            "command": "python mnist_cnn.py"
        }
    ]
}

In [ ]:
%%writefile mnist_v2.yaml
protocolVersion: 2
name: keras_tensorflow_mnist
type: job
version: 1.0
contributor: OpenPAI
description: |
  # Keras Tensorflow Backend MNIST Digit Recognition Examples
  Trains a simple convnet on the MNIST dataset.
  Gets to 99.25% test accuracy after 12 epochs
  (there is still a lot of margin for parameter tuning).
  16 seconds per epoch on a GRID K520 GPU.

  Reference https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

prerequisites:
  - protocolVersion: 2
    name: keras_tensorflow_example
    type: dockerimage
    version: 1.0
    contributor : OpenPAI
    description: |
      This is an [example Keras with TensorFlow backend Docker image on OpenPAI](https://github.com/Microsoft/pai/tree/master/examples/keras).
    uri : openpai/pai.example.keras.tensorflow

taskRoles:
  train:
    instances: 1
    completion:
      minSucceededInstances: 1
    dockerImage: keras_tensorflow_example
    resourcePerInstance:
      cpu: 4
      memoryMB: 8192
      gpu: 1
    commands:
      - python mnist_cnn.py

In [ ]:
tests = ["submit_v1", "submit_v2", "sub_oneliner"]
jobnames = {k: k + '_' + randstr().hex for k in tests}

options = ""
# options += " --preview"

if not in_job_container():
    jobs, cmds = [], []
    
    # submit v1
    jobs.append("submit_v1_" + randstr().hex)
    cmds.append(f'opai job submit {options} --update jobName={jobs[-1]} mnist_v1.json')

    # submit v2
    jobs.append("submit_v2_" + randstr().hex)
    cmds.append(f'opai job submit {options} --update name={jobs[-1]} mnist_v2.yaml')
    
    # sub
    jobs.append("sub_" + randstr().hex)    
    resource = '-i openpai/pai.example.keras.tensorflow --cpu 4 --memoryMB 8192 --gpu 1'
    cmds.append(f'opai job sub {options} -j {jobs[-1]} {resource} python mnist_cnn.py')

    # notebook
    jobs.append("notebook_" + randstr().hex)    
    cmds.append(f'opai job notebook {options} -j {jobs[-1]} {resource} --python python3 --pip-installs keras 2-submit-job-from-local-notebook.ipynb')

    for cmd in cmds:
        print(cmd, "\n")
        ! {cmd}
        print("\n")
    
    states = ClusterList().load().get_client(cluster_alias).wait(jobs)
    failed_jobs = [t for i, t in enumerate(jobs) if states[i] != "SUCCEEDED"]
    assert not failed_jobs, "some of jobs fails %s" % failed_jobs